<h1 style="text-align:center">Seafood Restaurant Business Analysis with Yelp Data </h1>
<h4 style="text-align:right">Pengfei He, Zejin Gao, Siqi Shen, Anne Huen Wai Wong</h4>

# 1 Introduction

## 1.1 Thesis Statement

In this project, we analyze yelp data focusing on the seafood restaurant business. Our goal is to explore factors extracted from attributes and reviews that could have an influence on business ratings. And furthermore, we provide useful, analytical suggestions to seafood business owners in order to improve their Yelp ratings. Our work can be mainly divided into two parts: attributes analysis and review analysis. For the first part, we keep the attributes that over half of the businesses have and do **imputation** with **Decision Tree** for these attributes. Then we fit an **multi-way ANOVA model** and find important attributes. For the second part, we do **sentiment analysis** for informative nouns in reviews. Then, we compute **correlation coefficient** between customer sentiment and business stars. We also do **Chi-Square Test** to test the indenpendence between customers' attitudes and different words.  Finally, we combine our findings of the two parts together and give our conclusions and suggestions.

## 1.2 Data Background

This Yelp dataset contains about 6.69 million reviews and about 193 thousand businesses from the following cities: Montreal (Canada), Waterloo (Canada), Pittsburgh (U.S.), Charlotte (U.S.), Urbana-Champaign (U.S.), Phoenix (U.S.), Las Vegas (U.S.), Madison (U.S.), Cleveland (U.S.). There are four JSON files.

* **review json** contains 6,685,900 reviews.
* **business.json** contains information about 192,609 businesses.
* **user.json** contains information about 1,637,138 users.
* **tip.json** contains information about 1,223,094 tips written by users on businesses.

# 2 Data Preprocessing

We filter business data following these rules:
1. **`"Seafood"`, `"Restaurant"` must in the category of the business.** This is due to our analysis goal.
2. **`"Steakhouse"` must not in the category of the business.** We random sample some restaurants that have both `"Seafood"` and `"Steakhouse"` in their categories and check their homepages on yelp. It is obvious that these restaurants focus on steak more than seafood, so we exclude `"Steakhouse"` when filtering.
3. **The length of the category must $\leq$ 50.** `"Seafood"`, `"Restaurant"` is a string that contains 19 characters; Set the total length of the category to $\leq$ 50 to make sure that there will not be too many characteristics for each restaurant, and we got restaurants that focus on seafood.
4. **The number of reviews must $\geq$ 50.** To avoid outliers in reviews, that is, to make our model more reliable for business, we select restaurants that have more reviews to build it. Count reviews for all businesses, we find 50 is a good amount.

We got **433** seafood businesses in total. Codes of filtering are in **Seafood Business Dataset.ipynb**. Dataset after filtered is in **seafood_business.csv**.

business_id|attributes|categories|name|review_count|stars
-----------|----------|----------|----|------------|-----
nsNONDHbV7Vudqh21uicqw|{'BusinessAcceptsCreditCards': 'True', 'Restau...|Seafood, Restaurants|Salt Cellar|526|3.5

After filtered business data, we combine reviews to the dataset. All of the reviews for seafood businesses are in **all_review.csv**.

# 3 Attribute Analysis

## 3.1 Attributes Selection

We count the attributes provided by these seafood businesses, which is **32** attributes in total. However, not all businesses provide 32 attributes; Some lack a lot and some provide enough. In order to avoid bias in our analysis, we sort all attributes and rule out those less than half of the seafood businesses provide. 

The following table shows attributes that more than half of the seafood businesses have:

Attribute| Counts | Attribute | Counts | Attribute | Counts | Attribute | Counts
------------|------|----------|--------|-----------|--------|-----------|--------
RestaurantsTableService|225|BikeParking|414|HasTV|432|RestaurantsGoodForGroups|433
BusinessAcceptsCreditCards|347|WiFi|426|RestaurantsDelivery|432|RestaurantsReservations|433
GoodForMeal|381|NoiseLevel|427|RestaurantsPriceRange2|432|BusinessParking|433
Caters|401|RestaurantsAttire|430|Ambience|432|RestaurantsTakeOut|433
||Alcohol|431|OutdoorSeating|432|GoodForKids|433

## 3.2 Missing Values

### 3.2.1 Manually Make Up Missing Values

Check the 4th column of the table above, we surprisingly find there are only 3 restaurants lack of 1 attribute. We search these 3 restaurants on Yelp and manually check their attributes to make up in our dataset. Thus, we have **10** attributes that all 433 seafood businesses provide. 

### 3.2.2 Imputation for Missing Values

To better analyze the relationship between attributes and star ratings, we need to impute missing values. Since all of our attributes are categorical data, we decide to use **Decision Tree**, which is an interpretable method and is easy to implement. 

**eg.** `BusinessAcceptCreditCard`  
It has 85 missing values and it only contains `True` or `False`. Because `Ambience` and `BusinessParking` have sublevels, which is hard to split at the tree node, we use the other **8** full attributes as input and the existing `BusinessAcceptCreditCard` as response to train the model. To evaluate the performance of this model, **Cross-Validation** is applied to obtain a score. Using 80% of the data as the training set, the accuracy on the testing set is 0.97, which is an incredibly good result. Therefore, the model we get could impute `BusinessAcceptCreditCard` well. 

We do similar processes to other attributes and finally obtain a full dataset **business_attributes.csv**.

## 3.3 Linear Regression and ANOVA

We consider **Linear Regression** and **ANOVA** to fit and test. We first fit a model including interactions. In this way, only `BusinessAcceptCreditCards`, `GoodForMeal`, `WiFi`, `NoiseLevel`, `RestaurantsAttire`, `Alcohol`, `RestaurantsGoodForGroups` and `GoodForKids` are significant. So we see these attributes as important ones and fit our final model by them.

ANOVA table as follows:

Attribute|Value|F-test p-value
---------|-----|--------------
NoiseLevel|average (base), loud (-0.196), quiet (+0.146), very_loud (-0.626)|0.0002
Alcohol|beer_and_wine (base), full_bar (-0.220), none (-0.008)|0.0003
WiFi|free (base), no (-0.181), paid (+0.467)|0.002
RestaurantsAttire|casual (base), dressy (+0.211)|0.005
GoodForKids|False (base), True (-0.191)|0.017
BusinessAcceptsCreditCardsTrue|False (base), True (+0.332)|0.026
RestaurantsGoodForGroups|False (base), True (-0.221)|0.041

# 4 Review Analysis

## 4.1 Data Cleaning

First, we obtain review data set with the features we need. The review data from "all_review.csv" contains 7 features. We only keep features: `business_id`, `stars` and `text`. Then we save the new review data set as **review_with_useful_features.csv**.

Second, we do **word tokenization**, which means we need to convert text into words. This process has **7 steps**: step1, convert "n't" to "not" and then connect "not" with the word after it, such as changing "wouldn't go" to "would not_go"; step2, break paragraph into words; step3, remove punctuation and abnormal string; step4, convert numbers to words; step5, convert words to lower case; step6, remove stopwords (we import "stopwords.words('english')" from python package nltk.corpus); step7, do lemmatization, such as changing "likes" to "like". 

An example is shown below.

business_id|stars|text|words
---------------------|------|--------------|-----------------------------
6xgcHeLad-VaoTIQewK84A|    3.0|  "Seems old and tired! And I wouldn't come again 3."|  \['seem','old','tired','would','not_come','three'\]


## 4.2 Sentiment Analysis

### 4.2.1 Positive and Negative Adjectives Classification

We use **Multinomial Naive Bayes Classifier** to classify adjectives in review text as positive and negative.

First, we define stars from 1 to 3 as negative and stars from 4 to 5 as positive. Then we convert stars into positive / negative tags and treat positive / negative tag as response variable. Second, we extract adjectives from tokenized words by python function "nltk.pos_tag(&nbsp;)" (part-of-speech tagging). Third, we count the frequency of each adjective in all review texts and obtain 1200 most frequent adjectives. Fourth, we count occurrences for these 1200 adjectives in each review text and obtain the frequency matrix with the index of review as row and 1200 adjectives as column. We treat this frequency matrix as the design matrix. Fifth, we fit the multinomial Naive Bayes model with design matrix and response variable. Sixth, we do prediction with each adjective and the positive / negative prediction result is the sentiment tag for adjectives.

**eg.** {'good': 'positive', 'delicious': 'positive', 'friendly': 'positive', 'bad': 'negative', 'decent': 'negative', 'slow': 'negative'}.

We save the dictionary for adjectives with positive / negative tags as **dict_adj.txt**.

### 4.2.2 Informative Nouns in Reviews

First, we extract nouns from tokenized words by python function "nltk.pos_tag(&nbsp;)" (part-of-speech tagging). Second, we count the frequency of each noun in all review texts. Because we assume that informative nouns would show up many times in review text, we only consider nouns with frequency larger than 4000. Third, we pick up informative nouns from them manually. 

**Informative nouns**: food, lobster, crab, shrimp, oyster, fish, clam; service, waiter, waitress, chef, manager; price.

### 4.2.3 Sentiment Analysis for Informative Nouns

From this part, we do counting at the restaurant level instead of review level. That is to say, we would count some kind of word from all the review texts of each restaurant respectively. Also, we need to obtain stars for each restaurant from "seafood_business.csv".

Then we obtain **sentiment table** for each informative nouns. First, we count the number of positive / negative adjectives in front of each informative noun. Second, we compute the proportion of positive adjectives among all the adjectives. For example, part of the sentiment table for food is shown below.

business_id|positive count|negative count|positive proportion|stars
-----------|--------------|--------------|-------------------|-----
nsNONDHbV7Vudqh21uicqw|102|15|0.872|3.5
F06m2yQSPHIrb1IT7heYeQ|70|1|0.986|4.0
W7hCuNdn2gzehta6eSHzgQ|9|10|0.474|2.0

We define the proportion of positive adjectives among all the adjectives as **customer sentiment**. This variable ranges from zero to one. zero means customers are completely unsatisfied, while one means customers are completely satisfied.

## 4.3 Tests and Results

First of all, we examine three main parts that could affect the seafood business stars. They are **food**, **service** and **price**. We count the total positive / negative adjectives in front of the three words and compute the customer sentiment. To know how much the customer sentiment influences stars, we compute the **correlation coefficient** between the restaurant stars and the customer sentiment. The result is shown in the table below.

&nbsp; |  positve count | nagative count | customer sentiment | correlation
-------|----------------|----------------|--------------------|-------------
food|24047|1907|0.927|0.547
service|15458|3162|0.830|0.580
price|7869|802|0.908|0.221

We treat the first two columns of the table as a contingency table. Then we do **Chi-Square Test** to see whether the three words and customer attitude are independent or not. The p-value is less than $2.2\times10^{-16}$, which means that customers have different attitudes towards food, service and price.

From the analysis above, we can find that food has the highest customer sentiment score, while service has the lowest. In terms of the correlation between customer sentiment and stars, customer sentiment of food and service have a strong correlation with star, while the correlation for price is not very strong. So we believe that food and service need to be further examined.

In **food category**, we find lobster, crab, shrimp, oyster, fish and clam. We do the same analysis except computing correlation. The table is shown below.

&nbsp; |  positve counts | nagative counts | customer sentiment
-------|-----------------|-----------------|-------------------
lobster|6022|374|0.942
crab|3146|479|0.868
shrimp|4379|265|0.942
oyster|802|58|0.948
fish|6646|558|0.923
clam|1154|100|0.920

The p-value for the chi-square contingency table test is less than $2.2\times10^{-16}$, which means that customers have different attitudes towards different kinds of food. Oyster has the highest customer sentiment score, while crab has the lowest.

In **service category**, we find waiter, waitress, chef and manager. We do the same analysis as the one above. The table is shown below.

&nbsp; |  positve counts | nagative counts | customer sentiment
-------|-----------------|-----------------|-------------------
waiter|6022|374|0.846
waitress|3146|479|0.834
chef|4379|265|0.945
manager|802|58|0.690

The p-value for the chi-square contingency table test is less than $2.2\times10^{-16}$, which means that customers have different attitudes towards different kinds of service. Chef has the highest customer sentiment score, while manager has the lowest.

# 5 Conclusions and Suggestions

## 5.1 Conclusions

From attribute analysis, the most significant attributes are `BusinessAcceptsCreditCards`, `WiFi`, `NoiseLevel`, `RestaurantsAttire`, `Alcohol`, `RestaurantsGoodForGroups` and `GoodForKids`. We also obtain the best combination for seafood businesses. A restaurant accepting credit cards, providing paid WiFi, having a quiet dining environment, asking for a dressy style, serving wine and beer, having fewer group customers and kids will receive the highest stars rating.

From review analysis, among **food**, **service**, and **price**, the seafood businesses need to improve service first. That is because customers' attitudes towards service influence the stars rating the most with a 0.580 correlation, while the current customers' attitudes towards service are the worst. Among the staff that provide service, only **69% of reviews** about the manager are associated with positive words, while **above 83% of reviews** about waiter / waitress / chef is associated with positive words. So the seafood businesses need to improve the service of their managers first. Besides, customers' attitudes towards food also influence the stars rating a lot with a 0.547 correlation. Among all kinds of seafood, only **87% of reviews** about crab is associated with positive words, while **above 92% of reviews** about other kinds of seafood are associated with positive words. So the seafood businesses need to improve the quality of crab most.



## 5.2 Suggestions

We recommend the seafood restaurant businesses:

1. Accept credit cards and provide WiFi.

2. Create a quiet dining environment and serve wine.

3. Serve fewer group customers and kids.

4. Improve the quality of service first and the quality of the seafood second.

5. Increase the manager's salary to hire a person who is very good at comforting angry guests because people have the worst evaluation towards the manager.

6. Put more effort to improve the quality of crab, since crab receives the most complaints among all kinds of seafood.

# 6 Strengths and Weaknesses

## 6.1 Strengths

1. The attributes model only contains significant attributes, making it simple and easy to interpret.

2. We extract customer sentiment from texts and get numeric data, thus, we could provide detailed and analytical insights to business owners.

3. We find further information about food and service so we could tell business owners what they should do in detail.

## 6.2 Weaknesses

1. The adjusted R squared of the attributes model is quite small, which means some variance could not be explained properly.

2. Since the decision tree is applied to impute missing values, it may potentially introduce some error.

3. There might exist adjective misclassification which will have an adverse effect on our sentiment analysis.

4. We only extract words but not phrases from the reviews, which might lead to information loss.

# 7 Contributions

* Pengfei He: code(data preprocessing, attribute analysis, EDA); image(attrubute analysis); summary(attribute analysis related part); presentation(attribute analysis)

* Anne Huen Wai Wong: code(data preprocessing, attribute analysis); image(attribute analysis and EDA); summary(data preprocessing, attribute analysis related part); presentation(attribute analysis)

* Zejin Gao: code(review analysis, EDA); image(review analysis and EDA); summary(introduction, review analysis related part); presentation(review analysis)

* Siqi Shen: code(review analysis, shiny); image(review analysis); summary(review analysis related part); presentation(review analysis, shiny)

# 8 References

[1] https://en.wikipedia.org/wiki/Seafood_restaurant

[2] https://www.geeksforgeeks.org/python-nlp-analysis-of-restaurant-reviews/

[3] https://www.kaggle.com/iwasdata/nltk-to-classify-adjectives-as-positive-negative
